# Face Identification

1. MTCNN으로 이미지 내 얼굴을 인식하고 얼굴 부분만 오려낸다.

2. 얼굴 이미지를  "FaceNet: A Unified Embedding for Face Recognition and Clustering"을 통해 특징 벡터로 표현한다.

3. 이 특징 벡터로 SVM 분류기를 학습하여 Face Identification 모델을 만든다.

## Import Libraries

현재 이 코드가 적힌 파일 위치에 facenet_pytorch이라는 폴더가 있고, 이를 모듈처럼 불러와 사용하는데,

이것도 `__init__.py`라는 파일이 있어야 이렇게 모듈처럼 불러올 수 있고, 동일 폴더 내에 있어야 하는 등 여러 조건이 있다.

In [1]:
from __future__ import print_function, division

# 구글이 공개한 facenet이라는 AI 모델이 있는데, tensorflow로 작성된 것을 pytorch 형태로 공개하고 있다.
# https://github.com/timesler/facenet-pytorch
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import os

# 경고 메시지 무시하기
import warnings
warnings.filterwarnings("ignore")

workers = 0 if os.name == 'nt' else 4

#### Determine if an nvidia GPU is available

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


## Import the Data

우리가 봤었던 MNIST의 경우, 데이터가 이미지인데 excel에 숫자로 적혀 있어서 pandas로 불러와서 이를 torch로 바꿔주었으나,

`featuresTrain = torch.from_numpy(features_train)`

`targetsTrain = torch.from_numpy(target_train).type(torch.LongTensor)`

`train = torch.utils.data.TensorDataset(featuresTrain,targetsTrain)`

여기서는 이미지 파일이기 때문에 torchvision.datasets이라는 내장된 모듈을 사용해 간편하게 불러온다.

***그리고 이미지를 align 해야 하는 과정이 필요하다.***

`dataset = datasets.ImageFolder(data_path)`

&nbsp;

***추가로, [Pytorch를 활용한 데이터 불러오는 과정에 대한 튜토리얼](https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html)***

## Pytorch를 사용하여 이미지를 불러올 때

1. 보통 `torchvision.datasets`로 데이터 객체를 선언하고, (`torchvision.datasets`은 `torch.utils.data.Dataset`의 서브 클래스이다)
2. `torch.utils.data.DataLoader`로 데이터 로드를 위한 객체를 선언하고,
3. 그 객체로 데이터를 로드한다.

데이터를 불러오는 과정에서 이러한 내장된 객체를 사용하는 이유는 여러 기능을 포함하기 때문이다.

`torchvision.datasets`의 경우,

- IMG_EXTENSIONS('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp')에 대해서 처리 가능하고,
- 데이터를 PIL(Python Imaging Library) 객체로 바뀌어 python 내에서 통일적으로 효율적으로 처리된다.
- 또한 이미지를 쉽게 변형시킬 수 있는 transform 기능을 포함하고 있다.
- 마지막으로 datasets 내에는 유명한 데이터셋을 쉽게 불러오거나 다운로드 받을 수 있고 있고, (`torchvision.datasets.MNIST`)
- 폴더 경로만 주어지면 자동으로 index를 포함하여 데이터셋을 만들 수 있는 객체도 포함한다. (`torchvision.datasets.ImageFolder`)

`torch.utils.data.DataLoader`의 경우 아래의 5가지 대표적인 기능을 하며,

- map-style and iterable-style datasets, (map은 각 데이터에 대해서 같은 처리를 동시에 하는 방식이고, iterable을 순회해가며 처리)
- customizing data loading order,
- automatic batching,
- single- and multi-process data loading,
- automatic memory pinning.

`torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, sampler=None, batch_sampler=None, num_workers=0, collate_fn=None, pin_memory=False, drop_last=False, timeout=0, worker_init_fn=None, multiprocessing_context=None)`

위를 보면, 다양한 parameter를 볼 수 있는데, parameter 변경으로 데이터를 로드할 때 여러 처리를 할 수 있다.

설명을 조금 하자면, Dataloader를 통해 batch로 혹은 non-batch로 데이터를 불러올 수 있는데, batch란 ***collating individual fetched data samples into batches***, 즉 개별 데이터를 batch 묶음 형태로 수집하는 것이다. Dataloader는 기본값으로 자동 batching을 지원한다.

개별 데이터의 index들을 사용하여 sample list로 묶이면, collate_fn 함수를 통해 batches로 수집(collate)된다.

```
for indices in batch_sampler:
    yield collate_fn([dataset[i] for i in indices])
```

중요한 것은,
The use of collate_fn is slightly different when automatic batching is enabled or disabled.

***When automatic batching is disabled***, collate_fn is called with each individual data sample, and the output is yielded from the data loader iterator. In this case, the default collate_fn simply converts NumPy arrays in PyTorch tensors.

***When automatic batching is enabled***, collate_fn is called with a list of data samples at each time. It is expected to collate the input samples into a batch for yielding from the data loader iterator. The rest of this section describes behavior of the default collate_fn in this case.

**Define a dataset and data loader**

We add the idx_to_class attribute to the dataset to enable easy recoding of label indices to identity names later one.

In [13]:
def collate_fn(x):
    return x[0]

dataset = datasets.ImageFolder("./data/train")
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)

In [14]:
dataset.samples[0]

('./data/train\\IU\\IU_ (1).JPG', 0)

In [15]:
for x, y in loader:
    print("x: ", x)
    print("y: ", y)
    break
    
#  collate_fn에 따른 결과값, 순서대로 x, x[0], x[0][0]
# [(<PIL.Image.Image image mode=RGB size=490x489 at 0x1AD3C10C7F0>, 0)]
# (<PIL.Image.Image image mode=RGB size=490x489 at 0x1AD3C58BDD8>, 0)
# <PIL.Image.Image image mode=RGB size=490x489 at 0x1AD3C56AE48>

x:  <PIL.Image.Image image mode=RGB size=490x489 at 0x21ECBF02788>
y:  0


#### Define MTCNN module

See `help(MTCNN)` for more details.

In [16]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709,
    device=device
)

In [22]:
aligned = []
indexes  = []
names = []
# x는 이미지(<PIL.Image.Image image mode=RGB size=490x489 at 0x1AD3C42EC50>)
# y는 index
for x, y in loader:
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        print('Face detected with probability: {:8f}'.format(prob))
        aligned.append(x_aligned)
        indexes.append(y)
        names.append(dataset.idx_to_class[y])

Face detected with probability: 0.998145
Face detected with probability: 0.999908
Face detected with probability: 0.999996
Face detected with probability: 0.998552
Face detected with probability: 0.993009
Face detected with probability: 0.999828
Face detected with probability: 0.999999
Face detected with probability: 0.999686
Face detected with probability: 0.999639
Face detected with probability: 0.999997
Face detected with probability: 0.999999
Face detected with probability: 0.999991
Face detected with probability: 0.999993
Face detected with probability: 0.999198
Face detected with probability: 0.999426
Face detected with probability: 0.999996
Face detected with probability: 0.999999
Face detected with probability: 0.999957
Face detected with probability: 0.999995
Face detected with probability: 0.999917
Face detected with probability: 0.999325
Face detected with probability: 0.999985
Face detected with probability: 0.998141
Face detected with probability: 0.999952
Face detected wi

In [26]:
resnet = InceptionResnetV1(pretrained='vggface2', classify=True).eval().to(device)

In [27]:
aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).detach().cpu()

In [28]:
embeddings[0]

tensor([3.4150, 0.7234, 0.3455,  ..., 2.2291, 2.9832, 3.0438])

In [29]:
indexes[0]

0

In [30]:
from sklearn import svm

In [32]:
clf = svm.SVC(kernel='linear', probability=True)
clf.fit(embeddings.tolist(), indexes)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

### Test

In [33]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709,
    device=device
)

In [34]:
resnet = InceptionResnetV1(pretrained='vggface2', classify=True).eval().to(device)

In [35]:
def collate_fn(x):
    return x[0]

dataset = datasets.ImageFolder("./data/test")
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)

In [44]:
aligned = []
indexes = []
names = []
for x, y in loader:
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        aligned.append(x_aligned)
        indexes.append(y)
        names.append(dataset.idx_to_class[y])

In [52]:
print(type(aligned))
print(type(aligned[0]))
print((aligned[0].size())

<class 'list'>
<class 'torch.Tensor'>
torch.Size([3, 160, 160])


In [53]:
aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).detach().cpu()

In [58]:
print(type(aligned))
print(aligned.size())

<class 'torch.Tensor'>
torch.Size([20, 3, 160, 160])


In [43]:
type(aligned)

torch.Tensor

In [60]:
embeddings[0].size()

torch.Size([8631])

In [39]:
result = clf.predict_proba(embeddings.tolist())

In [40]:
for r in result:
    print(r)

[0.36982908 0.03570551 0.06124283 0.06139958 0.09503324 0.07133033
 0.04158086 0.04414225 0.10127885 0.11845746]
[0.25093076 0.04572338 0.05509211 0.07343471 0.12484083 0.04618338
 0.04943881 0.04079879 0.10369446 0.20986278]
[0.03534779 0.50369624 0.03039686 0.09328613 0.04144831 0.04644801
 0.10265525 0.06475546 0.0276678  0.05429817]
[0.04517789 0.45934537 0.03736134 0.10340671 0.03867177 0.04670841
 0.12736922 0.06647125 0.0381958  0.03729225]
[0.04089995 0.13146285 0.41598478 0.05982932 0.08816559 0.06555806
 0.05564203 0.03384545 0.06986731 0.03874467]
[0.05297299 0.09824389 0.43164673 0.04939878 0.0907243  0.05401391
 0.05665086 0.06316787 0.05458602 0.04859465]
[0.04219221 0.06841136 0.02922624 0.47242545 0.03327145 0.07968511
 0.09455651 0.04205825 0.04159402 0.09657941]
[0.05290119 0.09549128 0.03395533 0.46642297 0.04394313 0.09319337
 0.07582471 0.04008945 0.04964924 0.04852934]
[0.06853329 0.05696401 0.05751299 0.03359082 0.49603691 0.06251467
 0.02629202 0.05572951 0.0258

마지막 결과는 확률 상 가능 높은 수치 값과 index를 의미하는데,

마지막 데이터셋은 일부러 오답을 넣어두었다. 윤하 폴더에 전소미 사진을 넣어두었다.

전소미는 그냥 한국인인데 외국인 느낌이 나서 섭외해봤다.

개인적인 호감은 없다.

데이터를 보면 알 수 있듯이, 가장 높은 확률이 30 ~ 50 사이로 큰 차이로 형성되는 것을 볼 수 있다.

가장 높은 확률을 정답으로 택하면 모두 맞히지만, 학습되지 않은 얼굴인 전소미에 대해서 21프로의 결과를 얻은 것을 보았을 때, 25프로 아래는 unknown으로 처리해야 할 수준이다.